<a href="https://colab.research.google.com/github/ArjunRAj77/Discharge-summary-extractor/blob/main/Discharge_Summary_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ngrok: filename=ngrok-0.0.1-py3-none-any.whl size=3060 sha256=563f6a9adcad806c1aa2a4079b812ce87a02a0e9317bec691dcd53e2c2765a0e
  Stored in directory: /root/.cache/pip/wheels/b2/f2/1e/11e57b35a7c1ea622a4cff4d9622bebd373f6371f3b1728a00
Successfully built ngrok


In [6]:
!./ngrok authtokens 2I25bavkwHYELR9tekrQDh8hpX7_e4JXyMTiF1UdcvBXdF7n

/bin/bash: ./ngrok: No such file or directory


In [7]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 36.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=c504dd977f82f55a003b38bd279c6ed6b05d126e36127a87babc3362349f2ba8
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(port='8501')
public_url

<NgrokTunnel: "http://3763-35-199-31-98.ngrok.io" -> "http://localhost:80">

In [5]:
pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install pyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 220 kB 9.4 MB/s 


In [3]:
pip install spacy_streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.3 MB 33.6 MB/s 
     |████████████████████████████████| 182 kB 33.0 MB/s 
     |████████████████████████████████| 164 kB 68.3 MB/s 
     |████████████████████████████████| 237 kB 69.6 MB/s 
     |████████████████████████████████| 78 kB 9.6 MB/s 
     |████████████████████████████████| 4.7 MB 92.6 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 51 kB 9.1 MB/s 
     |████████████████████████████████| 235 kB 82.4 MB/s 
     |████████████████████████████████| 235 kB 90.0 MB/s 
     |████████████████████████████████| 232 kB 97.1 MB/s 
     |████████████████████████████████| 231 kB 83.0 MB/s 
     |████████████████████████████████| 231 kB 75.3 MB/s 
     |████████████████████████████████| 231 kB 98.5 MB/s 
     |████████████████████████████████| 231 kB 93.2 MB/s 
     |██████████████████████████████

In [ ]:
pip install medspacy==0.2.0.1

In [31]:
%%writefile streamlit_app.py


# Importing libraries

import pandas as pd
import streamlit as st
import spacy_streamlit
import spacy
from spacy import displacy
from PyPDF2 import PdfReader
import medspacy
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent
from spacy_streamlit import visualize_parser


st.title('Discharge Summary Extractor')
uploaded_file = st.file_uploader("Choose a file")

@st.cache(allow_output_mutation=True)
def loading_ml_model(): 

  #problemlist=["allergies","asthma","diabetes"]
  #medilist=["Claritin","Zyrtec","Ortho Tri-Cyclen","Allegra"]

  #Getting disease list
  diseasedf = pd.read_csv('/content/diseasedataset.csv')
  diseaselist = diseasedf['diseasename'].tolist()
  print("Completed loading disease dataset.")

  # Getting drug list
  drugdf = pd.read_csv('/content/drugdataset.csv')
  druglist = drugdf['drugName'].tolist()
  dglist=[]

  # Here we are only using 2000 drugs names as input to maintain 
  for x in druglist[:2000]:
      dglist.append(x)
  print("Completed loading drug dataset.")

  
  # Load medspacy model

  nlp = medspacy.load()
  print(nlp.pipe_names)
  print("Started adding rules to nlp model..........")
  # Add rules for target concept extraction
  target_matcher = nlp.get_pipe("medspacy_target_matcher")
  for i in diseaselist:
    for j in dglist:
      target_rules = [
        TargetRule(i, "PROBLEM"),  
        TargetRule(j, "MEDICATION")
            ]
      target_matcher.add(target_rules)
  print("Completed making rules in nlp model")
  return nlp

def post_processing(doc):

    problem_label=[]
    medication_label=[]
    for ent in doc.ents:
        if ent.label_=='PROBLEM':
          problem_label.append(ent)
        if ent.label_=='MEDICATION':
          medication_label.append(ent) 
    print(problem_label)
    print(medication_label)
    prob_len=len(problem_label)
    med_len=len(medication_label)
    st.subheader("Summary:")
    st.write(f" Total number of identified diseases/problems in the file : {prob_len}")
    st.write(f" Total number of identified medications prescribed in the file : {med_len}")
    pass

if(uploaded_file is not None):
  reader = PdfReader(uploaded_file)
  print("Successfully read the file!")
  nlp=loading_ml_model()
  print("Completed loading the NLP model.")
  st.markdown('**Successfully loaded NLP model.**')
  if st.button('Extract'):
    st.subheader('Extracted Data:')
    st.markdown('Note: _The  first page of PDF is only considered for processing_!')
    number_of_pages = len(reader.pages)
    page = reader.pages[0]
    text_data = page.extract_text()

    # Processing Data with nlp model

    doc = nlp(text_data)
    print("Extracted the data.")

    # Adding colors to the rules
    colors = {"PROBLEM": "orange", "MEDICATION": "green"}
    options = {"colors": colors}
    visualize_ent(doc)
    print("Starting visualization of the data....")

    # Visualization of extracted data
    html =displacy.render(doc, style="ent", page=True,options=options)
    st.components.v1.html(html, width=1000, height=1000, scrolling=True)
    print("Visualization Completed.")
    print("Calling post processing function")
    post_processing(doc)
     

Overwriting streamlit_app.py


In [29]:
!streamlit run /content/streamlit_app.py & npx localtunnel --port 8501

[############......] - refresh-package-json:localtunnel: timing action:finalize

npx: installed 22 in 2.064s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.145.225.193:8501

your url is: https://forty-waves-thank-34-145-225-193.loca.lt
Successfully read the file!
Completed loading disease dataset.
Completed loading drug dataset.
['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context']
Started adding rules to nlp model..........
Completed making rules in nlp model
Completed loading the NLP model.
Successfully read the file!
Completed loading the NLP model.
/usr/local/lib/python3.7/dist-packages/medspacy/common/medspacy_matcher.py:80: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc)
Extracted the data.
<IPython.core.display.HTML object>
Starting visualization of the data....
Visualization Completed.
Calling post processing function
[Achilles ten